In [12]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
df = pd.read_csv("/Users/Abhirup/Desktop/Projects/HSBC DS Assignment/PS_20174392719_1491204439457_log.csv")
df.sort_values(by=['step'])
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})

In [3]:
df_cot=df[df['type'].isin(['CASH_OUT','TRANSFER'])]
df_cot=df_cot.reset_index(drop=True)
df_cot.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,TRANSFER,181.00,C1305486145,181.0,0.0,C553264065,0.0,0.00,1,0
1,1,CASH_OUT,181.00,C840083671,181.0,0.0,C38997010,21182.0,0.00,1,0
2,1,CASH_OUT,229133.94,C905080434,15325.0,0.0,C476402209,5083.0,51513.44,0,0
3,1,TRANSFER,215310.30,C1670993182,705.0,0.0,C1100439041,22425.0,0.00,0,0
4,1,TRANSFER,311685.89,C1984094095,10835.0,0.0,C932583850,6267.0,2719172.89,0,0


In [4]:
print(df_cot.shape)
print(len(df_cot['step'].drop_duplicates()))

(2770409, 11)
743


In [5]:
df_cot['step'].value_counts(bins=4).sort_index()

(0.257, 186.5]     990974
(186.5, 372.0]    1317246
(372.0, 557.5]     380545
(557.5, 743.0]      81644
Name: step, dtype: int64

In [6]:
#Since Steps are hours, make bins of observations to weeks
for i in range(4):
    df_cot.loc[(df_cot['step']>186*i) & (df_cot['step']<=186*(i+1)),'week'] =i+1
df_cot['week'].value_counts().sort_index()

1.0     990974
2.0    1317246
3.0     380559
4.0      81630
Name: week, dtype: int64

In [8]:
df_cot[df_cot['step']==187].head(1)

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week
990974,187,CASH_OUT,18166.68,C1133016021,10314.0,0.0,C1599052132,405951.81,424118.49,0,0,2.0


In [9]:
len(df_cot)

2770409

In [77]:
for i in range(1,4):
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+1))
    print(pd.merge(df_cot.loc[df_cot['week']==i,'nameOrig'],df_cot.loc[df_cot['week']==i+1,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())
    print("\nCustomers Transacting both in Week %s and Week %s"%(i,i+2))
    print(pd.merge(df_cot.loc[df_cot['week']==i,'nameOrig'],df_cot.loc[df_cot['week']==i+2,'nameOrig'],on='nameOrig')
      ['nameOrig'].nunique())


Customers Transacting both in Week 1 and Week 2
628

Customers Transacting both in Week 1 and Week 3
178

Customers Transacting both in Week 2 and Week 3
210

Customers Transacting both in Week 2 and Week 4
48

Customers Transacting both in Week 3 and Week 4
10

Customers Transacting both in Week 3 and Week 5
0


In [86]:
print(pd.merge(df_cot.loc[df_cot['week']==i,'nameOrig'],df_cot.loc[df_cot['week']==i+1,'nameOrig'],on='nameOrig')
      ['nameOrig'].head(10))

0    C1493903724
1     C698096077
2    C1002027777
3    C1735528721
4    C2091005120
5     C624219547
6     C515118379
7    C1866458559
8    C1170262181
9     C614844959
Name: nameOrig, dtype: object


In [93]:
df_cot[df_cot['nameOrig']=='C1866458559']

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week
2658054,522,CASH_OUT,117930.56,C1866458559,0.0,0.0,C2022053326,1841392.95,1959323.51,0,0,3.0
2689177,563,CASH_OUT,261870.19,C1866458559,0.0,0.0,C334675503,373186.47,635056.66,0,0,4.0


In [ ]:
df_cot_f=pd.DataFrame()
for i in range(990974,len(df_cot)):
    
    clear_output(wait=True)
    
    name=df_cot.iloc[i,3]
    step=df_cot.iloc[i,0]
    df_t1=df_cot.loc[(df_cot['step']>=step-186)&(df_cot['step']<step)&(df_cot['nameOrig']==name)
                    ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].sum()
    df_t2=df_cot.loc[(df_cot['step']>=step-186)&(df_cot['step']<step)&(df_cot['nameOrig']==name)
                     ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].count()
    
    df_t3=df_cot.iloc[[i]]
    
    if df_t1.empty==True:
        df_t3['$TransL1W']=0
        df_t3['NumTransL1W']=0
        df_cot_f=df_cot_f.append(df_t3)
    elif df_t1.empty==False:
        df_t1=df_t1.rename(columns={'amount':'$TransL1W'})
        df_t2=df_t2.rename(columns={'amount':'NumTransL1W'})
        df_t=pd.merge(df_t1,df_t2,on='nameOrig')
        df_t3=pd.merge(df_t3,df_t,how='left',on='nameOrig')
        df_cot_f=df_cot_f.append(df_t3)
        
    print("Current Progress:",np.round((i-990973)/(len(df_cot)-990973)*100,2),'%')

Current Progress: 7.75 %


In [ ]:
df_cot_f.to_csv('/Users/Abhirup/Desktop/Projects/HSBC DS Assignment/df_cot_f.csv', index=False)

In [16]:
df_cot_f[df_cot_f['NumTransL1W']>0]

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,$TransL1W,NumTransL1W
0,187,TRANSFER,180524.95,C1978304769,0.0,0.0,C1022973938,224968.76,405493.71,0,0,2.0,111781.51,1
0,187,CASH_OUT,19923.13,C73508477,0.0,0.0,C210689417,481992.87,501916.00,0,0,2.0,48611.48,1
0,187,CASH_OUT,218836.57,C1250432602,2050.0,0.0,C1333730561,318651.37,537487.94,0,0,2.0,121655.57,1
0,187,CASH_OUT,173802.67,C1613837471,1304.0,0.0,C973645281,759779.74,933582.41,0,0,2.0,534708.23,1
0,187,CASH_OUT,199991.30,C167211692,50795.0,0.0,C1838219587,0.00,199991.30,0,0,2.0,429669.13,1


In [17]:
df_cot[df_cot['nameOrig']=='C1978304769']

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week
387900,42,CASH_OUT,111781.51,C1978304769,0.0,0.0,C115283333,729560.77,841342.28,0,0,1.0
994782,187,TRANSFER,180524.95,C1978304769,0.0,0.0,C1022973938,224968.76,405493.71,0,0,2.0


In [111]:
#C1866458559
df_t1=df_cot.loc[(df_cot['step']>=563-186)&(df_cot['step']<563)&(df_cot['nameOrig']=='C1866458559')
                 ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].sum()
df_t2=df_cot.loc[(df_cot['step']>=563-186)&(df_cot['step']<563)&(df_cot['nameOrig']=='C1866458559')
                 ,['step','nameOrig','amount']].groupby('nameOrig',as_index=False)['amount'].count()

In [113]:
#df_t1=df_t1.to_frame()
df_t1=df_t1.rename(columns={'amount':'$TransL1W'})
df_t1

,nameOrig,$TransL1W
0,C1866458559,117930.56


In [114]:
#df_t2=df_t2.to_frame()
df_t2=df_t2.rename(columns={'amount':'NumTransL1W'})
df_t2

,nameOrig,NumTransL1W
0,C1866458559,1


In [115]:
df_t=pd.merge(df_t1,df_t2,on='nameOrig')
df_t

,nameOrig,$TransL1W,NumTransL1W
0,C1866458559,117930.56,1


In [125]:
df_t3=df_cot.iloc[2689177:2689177+1]
df_t3=pd.merge(df_t3,df_t,how='left',on='nameOrig')
df_t3

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,week,$TransL1W,NumTransL1W
0,563,CASH_OUT,261870.19,C1866458559,0.0,0.0,C334675503,373186.47,635056.66,0,0,4.0,117930.56,1


In [19]:
for i in range(1,4):
    print(i)

1
2
3
